# Train a Neural Network

In this notebook, you will train a Neural Network to drive a car around a track, using the data you collected in the previous lesson.

In [ ]:
import tarfile
import os, io
import random

import cv2
import numpy as np

import preprocessing

import matplotlib.pyplot as plt
%matplotlib inline

### Step 1: Load the training data

The code below assume you've copied the `data.tar.gz` file into the `Car-On-Track/` folder. If you haven't done this yet, go ahead and do this! Recall: The `data.tar.gz` file was what you created in the last lab -- it contains all the training data you collected manually.

In [ ]:
Xs = []
ys = []

def numpy_load_tar_file(f):
    b = io.BytesIO()
    b.write(f.read())
    b.seek(0)
    return np.load(b)

with tarfile.open("data.tar.gz", "r") as f:
    x_files = []

    for path in f.getnames():
        dirname, filename = os.path.split(path)
        if filename.startswith('X') and filename.endswith('.npy'):
            x_files.append(path)

    for x_file in x_files:
        y_file = x_file.replace('X', 'y')
        with f.extractfile(x_file) as fx:
            Xs.append(numpy_load_tar_file(fx))
        with f.extractfile(y_file) as fy:
            ys.append(numpy_load_tar_file(fy))

X = np.concatenate(Xs)
y = np.concatenate(ys)
print(X.shape)
print(y.shape)

### Step 2: Spot-check Your Data

The code below shows a random image (original and preprocessed). This is just to spot-check that we have read the data correctly, and that our preprocessing looks okay. You can run the code several times to see several different examples.

In [ ]:
rand_index = random.randint(0, len(X)-1)
print(rand_index)

x = preprocessing.crop(X[rand_index])
print(x.shape)
plt.imshow(x)

In [ ]:
x = preprocessing.edges(preprocessing.crop(X[rand_index]))
print(x.shape)
plt.imshow(x)

### Step 3: Preprocess all Training Images

In [ ]:
min_val, max_val, mid_val = -45.0, 45.0, 0.0    # y.min(), y.max(), (y.min() + y.max())/2.
print(min_val, max_val, mid_val)

In [ ]:
def preprocess_X(X):
    X_new = []
    for img in X:
        img_edge, img_feats = preprocessing.preprocess(img)
        X_new.append(img_feats)
    return np.array(X_new)

def preprocees_y(y):
    return (y - mid_val) / (max_val - min_val)

X_pcd = preprocess_X(X)
y_pcd = preprocees_y(y)
print(X_pcd.shape)
print(y_pcd.shape)

### Step 4: Spot-check the Preprocessing

In [ ]:
p = np.random.permutation(len(X))

In [ ]:
fig, axes = plt.subplots(3, 4, figsize=(15,15))

for img, img_pcd, label, ax in zip(X[p], X_pcd[p], y[p], axes.flatten()):
    img_pcd = np.array(cv2.cvtColor(img_pcd * 255.0, cv2.COLOR_GRAY2RGB), dtype=np.uint8)
    ax.imshow(np.concatenate((img, img_pcd), axis=0))
    ax.axis('off')
    ax.set_title(str(round(label, 2)))

### Step 5: Split the data into a "Training Set" and a "Test Set"

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_pcd, y_pcd, test_size=0.2)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

### Step 6: Augment the Data by Flipping the Frames Left-to-Right

We can double the amount of training data we have using a cleaver trick! (Remember: More data is better, and this essentially gives us more data for free.)

The trick is to flip each frame left-to-right (and also negate the label to match the new frame). One sample becomes two samples!

In [ ]:
def augment(X, y):
    X_new = []
    y_new = []
    for img, label in zip(X, y):
        X_new.append(img)
        y_new.append(label)
        X_new.append(np.fliplr(img))
        y_new.append(-label)
    return np.array(X_new), np.array(y_new)

X_train, y_train = augment(X_train, y_train)
X_test, y_test = augment(X_test, y_test)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
plt.imshow(cv2.cvtColor(np.concatenate((X_train[0], X_train[1]), axis=0), cv2.COLOR_GRAY2RGB))
plt.title("{}   {}".format(round(y_train[0], 2), round(y_train[1], 2)))

### Step 7: More Data Augmentation

This time we'll augment the data using a feature from Tensorflow which will randomly shift and stretch the images to create "synthetic" examples.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Convolution2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras import callbacks

In [ ]:
gen = ImageDataGenerator(featurewise_center=False,
                         samplewise_center=False,
                         featurewise_std_normalization=False,
                         samplewise_std_normalization=False,
                         zca_whitening=False,
                         rotation_range=5.0,
                         width_shift_range=0.05,
                         height_shift_range=0.05,
                         shear_range=0.0,
                         zoom_range=0.05,
                         channel_shift_range=0.0,
                         fill_mode='constant',
                         cval=0.0,
                         horizontal_flip=False,
                         vertical_flip=False,
                         rescale=None,
                         preprocessing_function=None)

fig, axes = plt.subplots(8, 4, figsize=(15, 15))

for X_batch, y_batch in gen.flow(X_train, y_train, batch_size=len(axes.flatten())):
    for img, label, ax in zip(X_batch, y_batch, axes.flatten()):
        ax.imshow(cv2.cvtColor(img, cv2.COLOR_GRAY2BGR))
        ax.axis('off')
        ax.set_title(str(round(label, 2)))
    break

### Step 8: Setup and Train a Convolutional Neural Network

Our network will have five layers:
- three convolutional layers (each followed by max-pooling),
- two fully connected layers (the first using dropout).

**NOTE:** This step will take several hours to run if you have a reasonable amount of data.

In [ ]:
img_in = Input(shape=X_train.shape[1:], name='img_in')
angle_in = Input(shape=(1,), name='angle_in')

x = Convolution2D(4, (3, 3))(img_in)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Convolution2D(8, (3, 3))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(3, 3))(x)

x = Convolution2D(16, (3, 3))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(3, 3))(x)

merged = Flatten()(x)

x = Dense(16)(merged)
x = Activation('linear')(x)
x = Dropout(.2)(x)

angle_out = Dense(1, name='angle_out')(x)

model = Model(inputs=[img_in], outputs=[angle_out])
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

In [ ]:
save_path = 'model_02.hdf5'

save_best = callbacks.ModelCheckpoint(save_path, monitor='val_loss', verbose=2,
                                      save_best_only=True)

early_stop = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10,
                                     verbose=2)

callbacks_list = [save_best, early_stop]

In [ ]:
batch_size = 128

model.fit(gen.flow(X_train, y_train, batch_size=batch_size),
          steps_per_epoch=len(X_train) // batch_size,
          epochs=500,
          validation_data=(X_test, y_test),
          callbacks=callbacks_list,
          verbose=2)

### Step 9: Load the Network from File & Spot-check It

In [ ]:
model = load_model('model_02.hdf5')

**Code below prints 25 random samples, using the NN to predict the steering angle, and also showing what the "correct" steering angle is.** This is useful to "spot-check" the performance. To see performance over the _entire_ test set, you can see the log above in the training phase.

In [ ]:
p = np.random.permutation(len(X_test))

fig, axes = plt.subplots(5, 5, figsize=(15,10))

for img, label, ax in zip(X_test[p], y_test[p], axes.flatten()):
    pred = round(model.predict(np.expand_dims(img, axis=0))[0][0], 2)
    ax.imshow(np.squeeze(img))
    ax.axis('off')
    ax.set_title(str(round(label, 2)) + "     " + str(pred))

**Code below prints the 25 _worst_ predictions by the NN.** This is useful to diagnose situation where the NN will do poorly. It is also nice to expose samples in your data which are _mislabeled_.

In [ ]:
predictions = model.predict(X_test)[:,0]
errors = np.abs(predictions - y_test)
indexes = np.argsort(errors)

fig, axes = plt.subplots(5, 5, figsize=(15,10))

for index, ax in zip(indexes[::-1], axes.flatten()):
    img = X_test[index]
    label = y_test[index]
    pred = predictions[index]
    ax.imshow(np.squeeze(img))
    ax.axis('off')
    ax.set_title(str(round(label, 2)) + "     " + str(round(pred, 2)))